In [1]:
#!/usr/bin/env python

def get_input(fname):
    with open(fname) as f:
        return [tuple(map(int, line.strip().split(", "))) for line in f.readlines()]

In [2]:
test_input = get_input("test.txt")
print(test_input)

[(1, 1), (1, 6), (8, 3), (3, 4), (5, 5), (8, 9)]


In [3]:
def manhattan_distance(x, y):
    return abs(x[0] - y[0]) + abs(x[1] - y[1])

In [4]:
import math
from collections import deque, Counter

def neighbors(p):
    n = []
    for delta in [(-1, 0), (0, -1), (1, 0), (0, 1)]:
        n.append((p[0] + delta[0], p[1] + delta[1]))
    return n

def get_areas(points):
    xmin, xmax = min(p[0] for p in points), max(p[0] for p in points)
    ymin, ymax = min(p[1] for p in points), max(p[1] for p in points)
    distances = [[(0, None) for _ in range(1 + ymax)] for _ in range(1 + xmax)]
    is_valid = lambda p: p[0] >= xmin and p[0] <= xmax and p[1] >= ymin and p[1] <= ymax
    q = deque()
    for i in range(len(points)):
        p = points[i]
        distances[p[0]][p[1]] = (0, {i})
        for n in neighbors(p):
            if is_valid(n):
                q.append(n)
    print((xmax - xmin + 1) * (ymax - ymin + 1))
    while len(q) > 0:
        p = q.popleft()
        if distances[p[0]][p[1]][1] is not None:
            continue
        neigh = neighbors(p)
        min_dist = math.inf
        min_idxs = None
        for neighbor in neigh:
            if not is_valid(neighbor):
                continue
            if distances[neighbor[0]][neighbor[1]][1] is None:
                q.append(neighbor)
            else:
                if distances[neighbor[0]][neighbor[1]][0] + 1 < min_dist:
                    min_dist = distances[neighbor[0]][neighbor[1]][0] + 1
                    min_idxs = set(distances[neighbor[0]][neighbor[1]][1])
                elif distances[neighbor[0]][neighbor[1]][0] + 1 == min_dist:
                    min_idxs |= distances[neighbor[0]][neighbor[1]][1]
        distances[p[0]][p[1]] = (min_dist, min_idxs)
    infinites = set()
    for i in range(xmin, 1+xmax):
        for j in (ymin, ymax):
            if len(distances[i][j][1]) == 1:
                infinites |= distances[i][j][1]
    for j in range(ymin, 1+ymax):
        for i in (xmin, xmax):
            if len(distances[i][j][1]) == 1:
                infinites |= distances[i][j][1]
    return [e[1].pop() for l in distances[xmin:1+xmax] for e in l[ymin:1+ymax] if len(e[1]) == 1 and len(e[1] & infinites) == 0]        

In [5]:
areas = get_areas(test_input)
counter = Counter(areas)
print(counter.most_common(1))

72
[(4, 17)]


In [6]:
actual_input = get_input("input.txt")
areas = get_areas(actual_input)
counter = Counter(areas)
print(counter.most_common(1))

98571
[(12, 4290)]


In [7]:
def get_region(points, threshold):
    xs = sorted([p[0] for p in points])
    ys = sorted([p[1] for p in points])
    start = (sum(xs) // len(xs), sum(ys) // len(ys))
    xmin, xmax = xs[0], xs[-1]
    ymin, ymax = ys[0], ys[-1]
    is_valid = lambda p: p[0] >= xmin and p[0] <= xmax and p[1] >= ymin and p[1] <= ymax
    q = deque()
    q.append(start)
    region = set()
    while len(q) > 0:
        c = q.popleft()
        if c in region:
            continue
        distance = sum(manhattan_distance(c, p) for p in points)
        if distance < threshold:
            region.add(c)
            for n in neighbors(c):
                if is_valid(n):
                    q.append(n)
    return region
        

In [8]:
region = get_region(test_input, 32)
print(len(region))

16


In [9]:
region = get_region(actual_input, 10000)
print(len(region))

37318
